由于分支定界法是 SCIP 的默认求解方法，因此不需要额外的设置。

## 生成数据的格式：  
1. 根据mps的node每个node生成一个对应的csv文件 里面包含变量名称+上界下界+全局上界下界  -----yes
2. 生成一个叫fixedvars的txt文件，如果这个node中有isFeasEQ的量就写进来var.name 如果没有不写==》之后可以merge
3. 每个node都生成一个叫BestSolution的csv文件，把每个node的BstSol都写进来 ----- no 
4. 每个问题都生成一个叫y_info的csv文件把每个运行了H的node的solver.getObjVal()（前+后）都写进来 ------ yes

Rins (Reduced Cost Fixing Heuristic)：这是SCIP的一个内置启发式方法，用于固定当前节点的整数变量。它根据变量的减少成本（reduced cost）来选择变量进行固定，以帮助加速求解过程。

In [4]:
from pyscipopt import Model, Eventhdlr, quicksum, SCIP_EVENTTYPE,SCIP_HEURTIMING
import pyscipopt
import sys
from tqdm import tqdm
import random
import pandas as pd

In [5]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE

class FixedVarsAtNode(Eventhdlr):
    """PySCIPOpt Event handler to write fixed vars of each node to a text file."""

    def __init__(self,random_seed):
        
        #define the path of file
        # self.output_var_info_file = "var_info/var_info.txt"
        
        #others
        self.df_y_info = pd.DataFrame(columns=['Before', 'After', 'y_label'])

        self.count=0
        self.count_limit=20
        
        #make random seed
        self.random_seed = random_seed
        self.random = random.Random(random_seed)

    def eventexec(self, event):
        if self.count<self.count_limit:
            
            # 创建一个空的DataFrame
            df_var_info = pd.DataFrame(columns=['Var', 'Type', 'LowerBound','UpperBound','GLB','GUB'])
            
            
            fixedvars=[]
            # get node number
            nodenumber = event.getNode().getNumber()
        
            #use_heuristic
            use_heuristic = self.random.choice([True, False])
            if use_heuristic:
                # 保存未使用启发式时的目标函数值
                obj_value_no_heuristic = self.model.getObjVal() 
    
                #写入node信息
                self.transvars = self.model.getVars(transformed=True)
                for var in self.transvars:
                    var_name = var.name
                    Glb = var.getUbGlobal()#获取变量的全局上界
                    Gub = var.getLbGlobal()#获取变量的全局下界
                    lb = var.getLbLocal()  # 获取变量的局部下界
                    ub = var.getUbLocal()  # 获取变量的局部上界
                    var_type = var.vtype()
                    
                    var_info_add_df ={'Var': var_name, 'Type': var_type, 'LowerBound': lb, 'UpperBound': ub, 'GLB':Glb, 'GUB':Gub}
                    #append to df_var_info
                    df_var_info = df_var_info.append(var_info_add_df, ignore_index=True)
                
                #输出为csv文件
                file_name='var_info/'+str(self.count) +"var_info.csv"
                df_var_info.to_csv(file_name, index=False) 
                del df_var_info
                
                #使用启发式后取得目标函数值
                self.model.setIntParam("heuristics/rins/freq",1)
                #self.model.includeHeur(rins_heuristic, "Rins", "Rins heuristic", "Y", timingmask=scip.SCIP_HEURTIMING.BEFORENODE)
                self.model.optimize() 
                
                obj_value_heuristic = self.model.getObjVal() 
                event.skip()
                if_y_better = self.check_if_better(obj_value_no_heuristic,obj_value_heuristic)
                
                #每个node运行后就直接存入对应的行，最后全运完直接把文件导出
                y_info_add_df ={'Node':self.count,'Before': obj_value_no_heuristic, 'After': obj_value_heuristic, 'y_label': if_y_better}
                self.df_y_info = self.df_y_info.append(y_info_add_df, ignore_index=True)
            else: 
                self.model.resetParam("heuristics/rins/freq")
                

            self.count+=1
        else:
            if self.count == self.count_limit+1:
                self.write_y_info_file('gen-ip002-test')
            pass

        
    def check_if_better(self,before,after):
        obj_fun=self.model.getObjectiveSense()
        
        if obj_fun == 'minimize':
            if before > after:
                if_better = 1
            else:
                if_better = 0
        else:
            if before < after:
                if_better = 1
            else:
                if_better = 0
        return if_better
            
        
    def write_y_info_file(self , filename):
        #输出为csv文件
        file_name = filename +"_y_info.csv"
        self.df_y_info.to_csv(file_name, index=False)
        

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.NODESOLVED, self)

    def eventexit(self):
        self.model.dropEvent(SCIP_EVENTTYPE.NODESOLVED, self)


In [ ]:
if __name__ == "__main__":
    # Specify the output file path
    #output_var_info_file = "var_info.txt"

    
    test = pyscipopt.scip.Model()
    test.readProblem("/Users/oukeikou/Desktop/sunruoyao/DATA/easy-sample/markshare_4_0.mps")
    print("read done")


    # Create and add event handler with the specified output file
    eventhdlr = FixedVarsAtNode(10)
    test.includeEventhdlr(eventhdlr, "FixedVarsAtNode", "Python event handler to write fixed variables after each solved node")

    # Optimize the problem
    test.optimize()
    print("optimized")
    
    eventhdlr.write_y_info_file('gen-ip002')

    # # ... 获取最优解等其他操作 ...
    # if test.getStatus() == "optimal":
    #     # 获取最优解的变量值
    #     optimal_solution = {var.name: test.getVal(var) for var in test.getVars()}

    #     # 输出最优解
    #     print("Optimal Solution:")
    #     for var_name, var_value in optimal_solution.items():
    #         print(f"{var_name}: {var_value}")
    #     node_count = test.getNNodes()
    #     print(node_count)

    # else:
    #         print("No optimal solution found~")

    # Free resources
    del test


original problem has 34 variables (30 bin, 0 int, 0 impl, 4 cont) and 4 constraints
read done
presolving:
(round 1, fast)       4 del vars, 0 del conss, 4 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 4 del conss, 4 add conss, 4 chg bounds, 4 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 3, exhaustive) 4 del vars, 4 del conss, 4 add conss, 4 chg bounds, 4 chg sides, 0 chg coeffs, 4 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 4 deleted vars, 4 deleted constraints, 4 added constraints, 4 tightened bounds, 0 added holes, 4 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3

In [4]:
            #通过使用 self.model.getVars(transformed=True)，你可以获取包括这些变换后的变量在内的所有变量的列表。
            #这些变量是经过模型内部变换后的版本，可能不同于你添加到模型中的原始变量。
#             self.transvars = self.model.getVars(transformed=True)
#             for var in self.transvars:
#                 var_name = var.name
#                 Glb = var.getUbGlobal()#获取变量的全局上界
#                 Gub = var.getLbGlobal()#获取变量的全局下界
#                 lb = var.getLbLocal()  # 获取变量的局部下界
#                 ub = var.getUbLocal()  # 获取变量的局部上界
#                 var_type = var.vtype()
#                 #self.model.getLocalTransEstimate()
#                 var_info_innode.append(f"Var: {var_name},Type:{var_type}, Lower Bound: {lb}, Upper Bound: {ub},Global Lower Bound: {Glb}, Global Upper Bound: {Gub}")

                #var : 这些变量对象通常包含变量的相关信息，例如名称、上界（upper bound）、下界（lower bound）、变量类型（例如整数变量、连续变量等）
                #self.transvars 是一个存储在事件处理器对象中的变量，用于保存模型中的所有变量
#                 if self.model.isFeasEQ(var.getLbLocal(), var.getUbLocal()):
#                     fixedvars.append(var.name)
                    ### 根本没进来
#             self.write_to_var_info_file(info=var_info_innode)


#             nodenumber = event.getNode().getNumber()
#             bst = self.model.getBestSol()

#             self.var_info.append(f"Node {nodenumber}: {', '.join(var_info_innode)}")
#             self.var_info.append(f"Node {nodenumber} : 'BST:{bst}")

            # Update the last node number if it's a new maximum
#             if nodenumber > self.last_node_number:
#                 self.last_node_number = nodenumber
#         else:
#             pass
#         self.count+=1

In [3]:
test = pyscipopt.scip.Model()
test.readProblem("/Users/oukeikou/Desktop/sunruoyao/easy-sample/gen-ip002.mps")
print("read done")

original problem has 41 variables (0 bin, 41 int, 0 impl, 0 cont) and 24 constraints
read done


In [3]:
# if __name__ == "__main__":
#     # Specify the output file path
#     output_file = "fixed_vars.txt"

#     scip = Model()
#     # 创建变量
#     x = {}
#     for i in range(1, 6):
#         x[i] = scip.addVar(f"x{i}", vtype="INTEGER")

#     # 设置优化目标函数
#     scip.setObjective(10 * x[1] + 6 * x[2] + 4 * x[3] + 5 * x[4] + 8 * x[5], sense="maximize")

#     # 添加约束
#     scip.addCons(2 * x[1] + 2 * x[2] + 6 * x[3] + 8 * x[4] + 5 * x[5] <= 40, name="c1")
#     scip.addCons(3 * x[1] + 5 * x[2] + 2 * x[3] + 1 * x[4] + 6 * x[5] <= 55, name="c2")
#     scip.addCons(5 * x[1] + 2 * x[2] + 4 * x[3] + 3 * x[4] + 7 * x[5] <= 25, name="c3")



#     # Create and add event handler with the specified output file
#     eventhdlr = FixedVarsAtNode(output_file)
#     scip.includeEventhdlr(eventhdlr, "FixedVarsAtNode", "Python event handler to write fixed variables after each solved node")

#     # Optimize the problem
#     scip.optimize()
    
#     eventhdlr.write_to_file()

#     # ... 获取最优解等其他操作 ...
#     if scip.getStatus() == "optimal":
#         # 获取最优解的变量值
#         optimal_solution = {var.name: scip.getVal(var) for var in scip.getVars()}

#         # 输出最优解
#         print("Optimal Solution:")
#         for var_name, var_value in optimal_solution.items():
#             print(f"{var_name}: {var_value}")
#         node_count = scip.getNNodes()
#         print(node_count)

#     else:
#             print("No optimal solution found~")

#     # Free resources
#     del scip


In [5]:
# 打印问题信息，包括问题名称
print("约束条件数目:", test.getNConss())
print("变量数目:", test.getNVars())
print("目标函数类型:", test.getObjectiveSense())
print("分支策略:", test.getParam("branching/vanillafullstrong/priority"))

约束条件数目: 24
变量数目: 41
目标函数类型: minimize
分支策略: -2000
